# Общие данные

### Код без функций

In [8]:
import logging

from aiogram.contrib.fsm_storage.memory import MemoryStorage
from aiogram.dispatcher import FSMContext
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram import Bot, Dispatcher, executor, types
import markup as nav
from aiogram.types import InlineKeyboardMarkup, InlineKeyboardButton


class FSMUrl(StatesGroup):
    autoruurl = State()


TOKEN = "5421953900:AAEtg_wMMxbpf-U3t9iZTBVXN63B_io8Ivo"

bot = Bot(token=TOKEN)
dp = Dispatcher(bot, storage=MemoryStorage())

startmenutext = """
Привет, {0.first_name}!

Для продолжения пользования ботом воспользуйся кнопкой меню ниже!🚀
"""

menutext = """
Я - DueninCars бот 💰🚘.

Со мной ты можешь легко и удобно посчитать выгоду от покупки автомобиля в Республике Беларусь и продажи его в России.

Все, что нужно есть в меню ниже.
"""

helptext = """
*Нажимай меню и выбирай из 4 основных вариантов:*

1. *"3 в 1"* - в нем мы проанализируем данные исходя из 3 сайтов сразу
2. *Auto.ru* - тут ты получишь данные исключительно из этого портала
3. *Drom.ru* - тут ты получишь данные исключительно из этого портала
4. *Avito.ru* - тут ты получишь данные исключительно из этого портала
"""
autorutext = """
*Ты выбрал анализ цен по Auto.ru*.
 Для продолжения анализа тебе нужно найти интересующий тебя поддержанный автомобиль на Av.by и прислать ссылку на него в наш бот"
"""


@dp.message_handler(commands=['start'])
async def command_start(message: types.Message):
    await bot.send_message(message.from_user.id, startmenutext.format(message.from_user),
                           reply_markup=nav.mainMenu)


@dp.message_handler()
async def bot_menu(message: types.Message):
    if message.text == 'ℹ Меню':
        await bot.send_message(message.from_user.id, menutext.format(message.from_user), reply_markup=nav.menubuttons)


@dp.callback_query_handler(text='helptextquery')
async def help_command(callback: types.CallbackQuery):
    await callback.message.delete()
    await callback.message.answer(helptext, reply_markup=nav.helpbuttons, parse_mode="Markdown")
    await callback.answer()


@dp.callback_query_handler(text='menuhelp')
async def help_back(callback: types.CallbackQuery):
    await callback.message.delete()
    await callback.message.answer(menutext, reply_markup=nav.menubuttons)
    await callback.answer()


@dp.callback_query_handler(text='autoru')
async def autorucommand(callback: types.CallbackQuery):
    await callback.message.delete()
    await callback.message.answer(autorutext, reply_markup=nav.helpbuttons, parse_mode="Markdown")
    await FSMUrl.autoruurl.set()
    await callback.answer()

@dp.message_handler(state=FSMUrl.autoruurl)
async def auto_ru_url_function(message: types.Message, state: FSMContext):
    autourl = message.text
    await state.update_data(autoruurl=message.text)

    await state.finish()

if __name__ == '__main__':
    executor.start_polling(dp, skip_updates=True)


https://cars.av.by/ford/explorer/101993184
профессии ясная
None


### Библиотеки

In [2]:
from fake_useragent import UserAgent
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import math
import csv
from selenium import webdriver
import time
from seleniumwire import webdriver
from selenium.webdriver.common.by import By

### Необходимые словари для auto.ru

In [3]:
brand_name = {'Acura': 'acura', 'Alfa Romeo': 'alfa_romeo', 'Aston Martin': 'aston_martin',
 'Audi': 'audi', 'BMW': 'bmw', 'Bentley': 'bentley', 'Brilliance': 'brilliance',
 'Changan': 'changan', 'Chery': 'chery', 'Chevrolet': 'chevrolet', 'Citroen': 'citroen',
 'EXEED': 'exeed', 'FAW': 'faw', 'Ferrari': 'ferrari', 'Fiat': 'fiat', 'Ford': 'ford',
 'Geely': 'geely', 'Haval': 'haval', 'Honda': 'honda', 'Hyundai': 'hyundai',
 'Infiniti': 'infiniti', 'JAC': 'jac', 'Jeep': 'jeep', 'Kia': 'kia',
 'Lada': 'vaz', 'Lamborghini': 'lamborghini', 'MINI': 'mini', 'Maserati': 'maserati',
 'McLaren': 'mclaren', 'Mercedes-Benz': 'mercedes', 'Mitsubishi': 'mitsubishi',
 'Nissan': 'nissan', 'Opel': 'opel', 'Peugeot': 'peugeot', 'Porsche': 'porsche',
 'Renault': 'renault', 'SEAT': 'seat', 'Skoda': 'skoda', 'Smart': 'smart',
 'Suzuki': 'suzuki', 'Toyota': 'toyota', 'Volkswagen': 'volkswagen',
 'Volvo': 'volvo', 'Vortex': 'vortex', 'Zotye': 'zotye'}

transmission_name = {'автомат': 'AUTOMATIC', 'автоматическая': 'AUTOMATIC', 'робот': 'ROBOT',
                    'вариатор': 'VARIATOR', 'механика':'MECHANICAL'}

engine_group_name = {'бензин': 'GASOLINE', 'дизель': 'DIESEL', 'гибрид': 'HYBRID', 'электро': "ELECTRO"}

gear_type_name = {'передний': 'FORWARD_CONTROL', 'задний': 'REAR_DRIVE', "полный": 'ALL_WHEEL_DRIVE'}

param_auto_ru = {'brand': None, 'model': None, 'transmission': None, 'year': None,
         'engine_group': None,'gear_type': None,
         'displacement_from': None, 'displacement_to': None, 'price_from' : None,
        'price_to' : None, 'km_age_from' : None, 'km_age_to': None} 

# Непонятно как работает body_type_group, поэтому пока без него
# Кроме этого следует создать словари с моделями машин

# Скрапинг данных из av.by по ссылке

### Получение параметров av.by

In [4]:
def response_av_by(real_url):
    fake_ua = {'user-agent': UserAgent().random}
    response = requests.get(url=real_url, headers=fake_ua)
    
    brand, model = real_url.split('/')[3:-1]
    soupUrl = BeautifulSoup(response.text, 'lxml')

    params = str(soupUrl.find('div', class_='card__params'))
    params = re.sub('[·/;,]', '', params)

    counter = 0
    while counter != len(params):
        if params[counter]== '<':
            a = counter
            counter += 1
        elif params[counter] == '>':
            b = counter
            params = params[:a] + params[b + 1:]
            counter = 0
        else:
            counter += 1
    
    usd_price = soupUrl.find('div', class_='card__price-secondary')
    usd_price = re.sub('[≈$]', '', usd_price.text).split()
    usd_price = int(''.join(usd_price))

    params = params.split()
    year = int(params[0])
    transmission = params[2]
    displacement = int(float(params[3]) * 1000)
    fuel = params[5]
    km = int(params[6] + params[7])
    
    
    return usd_price, brand, model, year, transmission, displacement, fuel, km

# Auto.ru парсинг автомобилей

###  Функция входа на страницу

In [5]:
def response_auto_ru(url):
    parametrs_av_by = response_av_by(url)

    usd_price = parametrs_av_by[0]
    param_auto_ru['brand'] = parametrs_av_by[1]
    param_auto_ru['model'] = parametrs_av_by[2]
    param_auto_ru['year'] = parametrs_av_by[3]
    param_auto_ru['transmission'] = transmission_name[parametrs_av_by[4]]
    param_auto_ru['displacement_from'] = parametrs_av_by[5]
    param_auto_ru['displacement_to'] = parametrs_av_by[5]
    param_auto_ru['engine_group'] = engine_group_name[parametrs_av_by[6]]
    param_auto_ru['km_age_from'] = int(parametrs_av_by[7] * 0.5)
    param_auto_ru['km_age_to'] = int(parametrs_av_by[7] * 1.5)
    real_url =  f"https://auto.ru/cars/{param['brand']}/{param['model']}/{param['year']}-year/all" 
    del param['brand'], param['model'], param['year']
    response = requests.get(url=real_url, params = param)
    if response.status_code == 200:
        if param['transmission'] == 'AUTOMATIC':
            return base_from_auto_ru(str(response.url) + '&transmission=ROBOT&transmission=VARIATOR')
        else:
            return base_from_auto_ru(response.url)
    else:
        print(response.status_code)
        return 'Сайт не работает'

### Функция парсинга

In [6]:
def base_from_auto_ru(url):
    kms, names, links, prices = [], [], [], []
    with webdriver.Chrome() as browser:
        browser.get(url)
        if 'showcaptcha' in str(browser.current_url):
            browser.find_element(By.CLASS_NAME, "CheckboxCaptcha").click()
            time.sleep(1)
            url_of_captcha  = browser.find_element(By.CLASS_NAME, "AdvancedCaptcha-Image").get_attribute('src')
            with open('captchaname.png', 'wb') as file:
                request_url_of_captcha = requests.get(url_of_captcha).content
                file.write(request_url_of_captcha)
            captcha = input()
            browser.find_element(By.CLASS_NAME, "Textinput-Control").send_keys(captcha)
            browser.find_element(By.XPATH, "/html/body/div[1]/div/div/form/div/div[3]/button[3]").click()
            time.sleep(1)

        soupUrl = BeautifulSoup(browser.page_source, 'lxml')
        for i in soupUrl.find('span', class_='ButtonWithLoader__content').text.split():
            if i.isdigit():
                pages = math.ceil(int(i) / 37)


        for car in soupUrl.find_all('div', class_='ListingItem'):
            try:
                km = ''.join(car.find('div', class_='ListingItem__kmAge').text.split()[:-1])
                kms.append(km)
            except:
                km = None
                kms.append(km)
            try:
                name = car.find('a', class_='Link ListingItemTitle__link').text
                names.append(name)
            except:
                name = None
                names.append(name)
            try:
                link = car.find('a', class_='Link ListingItemTitle__link').get('href')
                links.append(link)
            except:
                link = None
                links.append(link)
            try:
                price = car.find('div', class_='ListingItem__priceBlock').text
                if price.find('₽', 1) + 1 == len(price):
                    price = int(''.join(price.split()[:-1]))
                else:
                    dm = []
                    p = str()
                    for x in price:
                        if x.isdigit():
                            p += x
                        elif x == '₽':
                            dm.append(int(p))
                            p = str()
                    price = dm
                prices.append(price)
            except:
                try:
                    price = car.find('a', class_='Link ListingItemPrice__link').text
                    price = int(''.join(price.split()[:-1]))
                except:
                    price = None
                prices.append(price)



        for i in range(2, pages + 1):
            browser.find_element(By.XPATH, f"/html/body/div[2]/div/div[2]/div[3]/div[2]/div/div[2]/div/div[7]/div/span/a[{i}]/span/span").click()    
            time.sleep(5)
            if 'showcaptcha' in str(browser.current_url):
                browser.find_element(By.CLASS_NAME, "CheckboxCaptcha").click()
                time.sleep(1)
                url_of_captcha  = browser.find_element(By.CLASS_NAME, "AdvancedCaptcha-Image").get_attribute('src')
                with open('captchaname.png', 'wb') as file:
                    request_url_of_captcha = requests.get(url_of_captcha).content
                    file.write(request_url_of_captcha)
                captcha = input()
                browser.find_element(By.CLASS_NAME, "Textinput-Control").send_keys(captcha)
                browser.find_element(By.XPATH, "/html/body/div[1]/div/div/form/div/div[3]/button[3]").click()
                time.sleep(1)


            s = BeautifulSoup(browser.page_source, 'lxml')

            for car in s.find_all('div', class_='ListingItem'):
                try:
                    km = ''.join(car.find('div', class_='ListingItem__kmAge').text.split()[:-1])
                    kms.append(km)
                except:
                    km = None
                    kms.append(km)
                try:
                    name = car.find('a', class_='Link ListingItemTitle__link').text
                    names.append(name)
                except:
                    name = None
                    names.append(name)
                try:
                    link = car.find('a', class_='Link ListingItemTitle__link').get('href')
                    links.append(link)
                except:
                    link = None
                    links.append(link)
                try:
                    price = car.find('div', class_='ListingItem__priceBlock').text
                    if price.find('₽', 1) + 1 == len(price):
                        price = int(''.join(price.split()[:-1]))
                    else:
                        dm = []
                        p = str()
                        for x in price:
                            if x.isdigit():
                                p += x
                            elif x == '₽':
                                dm.append(int(p))
                                p = str()
                        price = dm
                    prices.append(price)
                except:
                    try:
                        price = car.find('a', class_='Link ListingItemPrice__link').text
                        price = int(''.join(price.split()[:-1]))
                    except:
                        price = None
                    prices.append(price)
        browser.close()

        bd = {'Name': names, 'Price': prices, 'Km': kms, 'Link': links}
        autorubd = pd.DataFrame(bd)
        dictreal = []
        dictdiscountprice = []

        for i in autorubd.Price:
            if isinstance(i, int):
                dictreal.append(int(i))
                dictdiscountprice.append(None)
            elif i != None and len(i) == 2:
                dictreal.append(int(i[1]))
                dictdiscountprice.append(int(i[0]))
            elif i != None and len(i) == 1:
                dictreal.append(int(i[0]))
                dictdiscountprice.append(None)
            else:
                dictreal.append(None)
                dictdiscountprice.append(None)
        autorubd = autorubd.drop('Price', axis=1)
        autorubd['Real_price'] = dictreal
        autorubd['Discount_price'] = dictdiscountprice
        autorubd.to_csv('database_autoru.csv')
        
        return 'Загрузка базы данных завершена'

# Drom.ru парсинг автомобилей

###  Функция входа на страницу

### Функция парсинга

# Drom.ru парсинг автомобилей

###  Функция входа на страницу

### Функция парсинга